# loop on the OCs to estimate isochrone and mass

In [5]:
using PyCall
using Distributions, Statistics, Distances
using DataFrames
using Random , Printf, Glob

import PyPlot , CSV
import Distances, StatsBase

rootdir = "/home/stephane/Science/cluster/GAIA"

push!(LOAD_PATH,"$rootdir/master/src")
using GaiaClustering

## directory
wdir    = "$rootdir/products"
plotdir = "$rootdir/products/plotdens"
ocdir= "$wdir/oc"

cd(wdir)

In [6]:
cd(ocdir)
files= glob("*csv")
cd(wdir)

sample= rootdir*"/master/notebooks/data/BrowseTargets.18292.1530479692.fake"
df1= CSV.read(sample, delim= "|")
df1.name= strip.(df1.name)

println(df1)

noc= 0
for f in files
    fname= f[1:end-4] ; fvot= f[1:end-7]
    oc= CSV.read("$ocdir/$fname.csv" , delim= ";")
    ocfile= "$ocdir/$fname.cs" ; voname= "$fvot.vot"
    
end
   
        
        

1155×12 DataFrame
│ Row  │ Column1 │ name              │ ra       │ dec      │ cluster_radius │ rad_vel  │ rad_vel_error │ num_cluster_stars │ distance │ log_age │ log_age_error │ Column12 │
│      │ Missing │ SubString{String} │ String   │ String   │ Float64        │ String   │ String        │ Int64             │ Int64    │ Float64 │ String        │ Missing  │
├──────┼─────────┼───────────────────┼──────────┼──────────┼────────────────┼──────────┼───────────────┼───────────────────┼──────────┼─────────┼───────────────┼──────────┤
│ 1    │ missing │ MWSC 4688         │ 23 51 54 │ -86 43.2 │ 0.185          │          │               │ 57                │ 1336     │ 9.39    │               │ missing  │
│ 2    │ missing │ MWSC 5684         │ 12 53 43 │ -86 38.9 │ 0.155          │          │               │ 52                │ 1432     │ 9.18    │         0.023 │ missing  │
│ 3    │ missing │ MWSC 4176         │ 14 27 18 │ -85 25.2 │ 0.28           │          │               │ 155         

│ 47   │ missing │ NGC 4463          │ 12 29 56 │ -64 47.4 │ 0.14           │   -24.52 │          6.37 │ 133               │ 1115     │ 7.97    │               │ missing  │
│ 48   │ missing │ Loden 615         │ 12 20 41 │ -64 46.5 │ 0.245          │          │               │ 150               │ 826      │ 8.63    │               │ missing  │
│ 49   │ missing │ FSR 1678          │ 14 44 35 │ -64 44.4 │ 0.14           │          │               │ 85                │ 1791     │ 9.0     │               │ missing  │
│ 50   │ missing │ Ruprecht 98       │ 11 58 52 │ -64 35.3 │ 0.28           │    -7.60 │          1.70 │ 131               │ 610      │ 8.8     │               │ missing  │
│ 51   │ missing │ Loden 848         │ 13 34 23 │ -64 33.0 │ 0.225          │          │               │ 162               │ 1797     │ 8.75    │               │ missing  │
│ 52   │ missing │ ASCC 82           │ 15 47 02 │ -64 24.7 │ 0.21           │          │               │ 135               │ 800      │

│ 94   │ missing │ Loden 624         │ 12 26 19 │ -61 48.0 │ 0.32           │     8.00 │               │ 999               │ 1423     │ 7.6     │               │ missing  │
│ 95   │ missing │ FSR 1669          │ 13 56 17 │ -61 45.0 │ 0.12           │          │               │ 140               │ 1105     │ 8.0     │               │ missing  │
│ 96   │ missing │ ESO 137-23        │ 16 24 31 │ -61 44.0 │ 0.13           │          │               │ 85                │ 1561     │ 9.55    │               │ missing  │
│ 97   │ missing │ NGC 3766          │ 11 36 18 │ -61 36.9 │ 0.23           │   -14.80 │          1.72 │ 366               │ 1685     │ 7.95    │               │ missing  │
│ 98   │ missing │ Ruprecht 105      │ 12 33 54 │ -61 34.5 │ 0.195          │          │               │ 312               │ 1250     │ 9.1     │               │ missing  │
│ 99   │ missing │ FSR 1668          │ 13 54 54 │ -61 34.5 │ 0.14           │          │               │ 216               │ 1184     │

│ 165  │ missing │ vdBergh-Hagen 155 │ 13 57 30 │ -59 34.8 │ 0.13           │          │               │ 165               │ 1864     │ 8.44    │         0.102 │ missing  │
│ 166  │ missing │ ESO 130-08        │ 12 10 19 │ -59 31.2 │ 0.2            │   -38.74 │          7.92 │ 231               │ 800      │ 9.01    │         0.032 │ missing  │
│ 167  │ missing │ Ruprecht 113      │ 15 57 32 │ -59 29.7 │ 0.45           │          │               │ 756               │ 810      │ 8.345   │         0.215 │ missing  │
│ 168  │ missing │ ASCC 53           │ 09 38 10 │ -59 27.6 │ 0.265          │          │               │ 278               │ 1786     │ 8.0     │               │ missing  │
│ 169  │ missing │ Loden 1152        │ 13 59 28 │ -59 19.8 │ 0.145          │          │               │ 275               │ 1711     │ 8.065   │         0.092 │ missing  │
│ 170  │ missing │ ESO 130-06        │ 12 07 52 │ -59 18.0 │ 0.19           │    -0.96 │          5.75 │ 199               │ 1198     │

│ 211  │ missing │ Collinder 292     │ 15 50 02 │ -57 38.3 │ 0.17           │          │               │ 141               │ 1106     │ 8.7     │         0.062 │ missing  │
│ 212  │ missing │ FSR 1583          │ 11 31 40 │ -57 37.4 │ 0.12           │          │               │ 66                │ 1639     │ 8.6     │               │ missing  │
│ 213  │ missing │ NGC 5715          │ 14 43 30 │ -57 34.0 │ 0.115          │          │               │ 126               │ 1490     │ 8.97    │         0.028 │ missing  │
│ 214  │ missing │ Trumpler 19       │ 11 14 15 │ -57 32.1 │ 0.215          │          │               │ 210               │ 1968     │ 9.65    │               │ missing  │
│ 215  │ missing │ Ruprecht 91       │ 10 47 38 │ -57 28.2 │ 0.135          │     5.44 │          5.75 │ 61                │ 1095     │ 8.405   │         0.098 │ missing  │
│ 216  │ missing │ FSR 1621          │ 12 39 23 │ -57 22.8 │ 0.145          │          │               │ 92                │ 1015     │

...

│ 953  │ missing │ Cyg OB2           │ 20 33 16 │ +41 19.2 │ 0.425          │   -20.85 │          2.11 │ 904               │ 1593     │ 6.72    │               │ missing  │
│ 954  │ missing │ Dolidze 11        │ 20 26 28 │ +41 25.8 │ 0.1            │   -14.00 │               │ 60                │ 1813     │ 7.35    │               │ missing  │
│ 955  │ missing │ FSR 0236          │ 20 32 37 │ +41 26.4 │ 0.18           │          │               │ 303               │ 901      │ 8.2     │               │ missing  │
│ 956  │ missing │ NGC 7024          │ 21 06 13 │ +41 29.7 │ 0.155          │          │               │ 173               │ 1513     │ 8.425   │         0.052 │ missing  │
│ 957  │ missing │ FSR 0697          │ 04 18 41 │ +41 31.6 │ 0.115          │    -8.30 │          3.70 │ 64                │ 1600     │ 8.915   │         0.118 │ missing  │
│ 958  │ missing │ Dolidze 44        │ 20 29 47 │ +41 38.1 │ 0.2            │          │               │ 105               │ 1499     │

│ 1000 │ missing │ NGC 6811          │ 19 37 22 │ +46 23.7 │ 0.2            │     6.05 │          0.95 │ 138               │ 1227     │ 8.805   │         0.017 │ missing  │
│ 1001 │ missing │ Basel 13          │ 21 12 36 │ +46 31.8 │ 0.145          │          │               │ 209               │ 1313     │ 8.96    │         0.032 │ missing  │
│ 1002 │ missing │ NGC 7209          │ 22 04 54 │ +46 33.1 │ 0.23           │    -9.70 │               │ 258               │ 1250     │ 8.629   │               │ missing  │
│ 1003 │ missing │ ASCC 23           │ 06 20 21 │ +46 40.5 │ 0.375          │          │               │ 96                │ 600      │ 8.485   │               │ missing  │
│ 1004 │ missing │ ASCC 119          │ 22 18 40 │ +46 53.1 │ 0.21           │          │               │ 95                │ 1200     │ 8.791   │         0.017 │ missing  │
│ 1005 │ missing │ NGC 7082          │ 21 28 48 │ +47 06.6 │ 0.325          │    -9.10 │               │ 832               │ 1300     │

│ 1046 │ missing │ Kronberger 84     │ 21 35 38 │ +53 29.3 │ 0.09           │          │               │ 107               │ 1310     │ 8.825   │               │ missing  │
│ 1047 │ missing │ FSR 0656          │ 03 58 52 │ +53 38.4 │ 0.18           │          │               │ 219               │ 1400     │ 8.895   │         0.021 │ missing  │
│ 1048 │ missing │ FSR 0314          │ 21 27 52 │ +53 44.5 │ 0.14           │          │               │ 221               │ 1710     │ 8.14    │               │ missing  │
│ 1049 │ missing │ FSR 0317          │ 21 32 37 │ +53 45.3 │ 0.095          │          │               │ 118               │ 1934     │ 8.8     │               │ missing  │
│ 1050 │ missing │ ASCC 114          │ 21 40 01 │ +53 57.9 │ 0.16           │   -22.00 │          4.90 │ 105               │ 590      │ 7.75    │               │ missing  │
│ 1051 │ missing │ IC 1442           │ 22 15 59 │ +53 59.7 │ 0.155          │   -63.50 │          0.90 │ 151               │ 1750     │

│ 1093 │ missing │ Czernik 9         │ 02 33 32 │ +59 52.5 │ 0.09           │          │               │ 71                │ 1900     │ 8.75    │               │ missing  │
│ 1094 │ missing │ NGC 7429          │ 22 55 53 │ +59 57.6 │ 0.125          │          │               │ 118               │ 1250     │ 8.08    │               │ missing  │
│ 1095 │ missing │ Stock 23          │ 03 16 55 │ +60 00.9 │ 0.28           │   -16.75 │          4.25 │ 179               │ 450      │ 8.3     │               │ missing  │
│ 1096 │ missing │ NGC 433           │ 01 15 12 │ +60 07.4 │ 0.14           │   -36.91 │          0.18 │ 151               │ 1650     │ 7.81    │               │ missing  │
│ 1097 │ missing │ NGC 743           │ 01 58 37 │ +60 09.4 │ 0.17           │    -6.50 │          4.50 │ 215               │ 1100     │ 8.29    │         0.090 │ missing  │
│ 1098 │ missing │ NGC 129           │ 00 30 10 │ +60 12.2 │ 0.155          │   -38.03 │          0.82 │ 240               │ 1685     │

│ 1143 │ missing │ FSR 0519          │ 00 52 29 │ +64 40.0 │ 0.14           │          │               │ 95                │ 1517     │ 7.35    │               │ missing  │
│ 1144 │ missing │ ASCC 127          │ 23 08 50 │ +64 50.7 │ 0.675          │    -8.24 │          2.65 │ 427               │ 400      │ 7.82    │               │ missing  │
│ 1145 │ missing │ FSR 0516          │ 00 50 33 │ +65 57.9 │ 0.17           │          │               │ 65                │ 810      │ 8.83    │         0.052 │ missing  │
│ 1146 │ missing │ Berkeley 59       │ 00 02 14 │ +67 25.5 │ 0.22           │   -12.50 │          7.08 │ 89                │ 1000     │ 6.1     │               │ missing  │
│ 1147 │ missing │ Cep OB4           │ 00 02 56 │ +67 30.0 │ 0.76           │     0.00 │               │ 654               │ 850      │ 6.1     │               │ missing  │
│ 1148 │ missing │ NGC 7762          │ 23 49 52 │ +68 01.5 │ 0.213          │          │               │ 276               │ 780      │